<h1>Creating Table for Ranking<h1>

In [2]:
from pyspark.sql import SparkSession, functions as F

spark = (
    SparkSession.builder.appName("Ranking Features")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/25 21:09:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [28]:
transaction_data = spark.read.parquet('../data/curated/cleaned_data.parquet')
merchant_fraud_rates = spark.read.csv('../data/curated/merchant_fraud_rate.csv', header=True)
consumer_fraud_rates = spark.read.csv('../data/curated/consumer_fraud_rate.csv', header=True)
future_profit = spark.read.csv('../data/curated/future_predictions.csv', header=True)

In [29]:
transaction_data = transaction_data.join(consumer_fraud_rates, ['user_id'])

In [30]:
merchant_properties = transaction_data.groupBy("merchant_abn").agg(\
    F.avg('Median_age_persons').alias("avg_expected_customer_age"),\
    F.avg('fraud_rate').alias("avg_customer_fraud_rate")\
)
#comment

In [31]:
future_profit = future_profit.drop("_c0")

In [32]:
merchant_ranking_properties = merchant_properties.join(future_profit, ['merchant_abn']).join(merchant_fraud_rates, ['merchant_abn'])

In [35]:
merchant_ranking_properties.write.mode('overwrite').option('header', True).csv('../data/curated/merchant_ranking_properties.csv')

22/09/26 07:00:54 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 7206560 ms exceeds timeout 120000 ms
22/09/26 07:00:54 WARN SparkContext: Killing executors is not supported by current scheduler.
